In [1]:

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns 
%matplotlib inline
import statsmodels.formula.api as smf
from statsmodels.api import qqplot

plt.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False


In [2]:
df = pd.read_csv("/home/piai/바탕화면/BigData_Project/Car.csv")
df.head()

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price
0,Maruti Wagon R LXI CNG,Mumbai,2682.68,2010,72000,CNG,Manual,First,26.6 kmpl,998 CC,58.16 bhp,5.0,NaN
1,Hyundai Creta 1.6 CRDi SX Option,Pune,19162.00,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN
2,Honda Jazz V,Chennai,6898.32,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh
3,Maruti Ertiga VDI,Chennai,9197.76,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,27194.71,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN


In [3]:
df.drop("New_Price" , axis = 1, inplace =True)
df_raw = df.dropna(subset = ["Price"] , axis = 0).reset_index(drop = True)
df_raw.head()

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti Wagon R LXI CNG,Mumbai,2682.68,2010,72000,CNG,Manual,First,26.6 kmpl,998 CC,58.16 bhp,5.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,19162.00,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0
2,Honda Jazz V,Chennai,6898.32,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0
3,Maruti Ertiga VDI,Chennai,9197.76,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,27194.71,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0


In [4]:
len(df_raw)

6200

In [5]:
df_raw.isna().sum()

Name                  0
Location              0
Price                 0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               2
Engine               36
Power                36
Seats                42
dtype: int64

In [6]:
df_raw.head()
# 참고 사이트 - https://www.cardekho.com/overview/Ford_Fiesta_2008-2011/Ford_Fiesta_2008-2011_1.4_SXi_TDCi_ABS.htm

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti Wagon R LXI CNG,Mumbai,2682.68,2010,72000,CNG,Manual,First,26.6 kmpl,998 CC,58.16 bhp,5.0
1,Hyundai Creta 1.6 CRDi SX Option,Pune,19162.00,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0
2,Honda Jazz V,Chennai,6898.32,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0
3,Maruti Ertiga VDI,Chennai,9197.76,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,27194.71,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0


In [7]:
# 파워와 엔진이 널인 행의 모델 출력
rows_with_nulls = df_raw[df_raw['Power'].isnull() & df_raw['Engine'].isnull()]
rows_with_nulls["Name"].unique()

array(['Honda City 1.5 GXI', 'Maruti Swift 1.3 VXi',
       'Land Rover Range Rover 3.0 D', 'Honda City 1.3 DX',
       'Maruti Swift 1.3 ZXI',
       'Land Rover Range Rover Sport 2005 2012 Sport',
       'Maruti Swift 1.3 LXI', 'Hyundai Santro LP zipPlus',
       'Toyota Etios Liva V', 'Maruti Swift 1.3 VXI ABS',
       'BMW 5 Series 520d Sedan', 'Hyundai Santro GLS II - Euro II',
       'Maruti Wagon R Vx', 'Honda CR-V AT With Sun Roof',
       'Fiat Punto 1.3 Emotion', 'Hyundai Santro Xing XG',
       'Honda Jazz Select Edition', 'Fiat Punto 1.2 Dynamic',
       'Fiat Punto 1.4 Emotion'], dtype=object)

In [8]:
# Engin, Power, Seats 값 채워넣기
dict1 = {"Honda City 1.5 GXI" : [1493.0, 100.0, 5],
 "Maruti Swift 1.3 VXi" : [1298.0, 87.0, 5],
 "Land Rover Range Rover 3.0 D" : [2997.0, 394.0, 7],
 "Honda City 1.3 DX" : [1298.0, 100.0, 5],
 "Maruti Swift 1.3 ZXI" : [1248.0, 88.5, 5],
 "Land Rover Range Rover Sport 2005 2012 Sport" : [2993.0, 241.4, 5],
 "Maruti Swift 1.3 LXI" : [1248.0 , 88.5, 5],
 "Hyundai Santro LP zipPlus" : [1086.0, 63.0, 5],
 "Toyota Etios Liva V" : [1197.0, 79.0, 5],
 "Maruti Swift 1.3 VXI ABS" : [1197.0, 83.0, 5],
 "BMW 5 Series 520d Sedan" : [1995.0, 187.0, 5],
 "Hyundai Santro GLS II - Euro II" : [1086.0, 63.0, 5],
 "Maruti Wagon R Vx" : [998.0, 67.0, 5],
 "Honda CR-V AT With Sun Roof" : [1498.0,  190, 5],
 "Fiat Punto 1.3 Emotion" : [1248.0, 75.0, 5],
 "Hyundai Santro Xing XG" : [1086.0, 63.0, 5],
 "Honda Jazz Select Edition" : [1498.0, 118.0, 5],
 "Fiat Punto 1.2 Dynamic" : [1242.0, 68.0, 5],
 "Fiat Punto 1.4 Emotion" : [1368.0, 90.0, 5]
 }

for key, value in dict1.items():
    df_raw.loc[df_raw["Name"] == key, "Engine"] = value[0]
    df_raw.loc[df_raw["Name"] == key, "Power"] = value[1]
    df_raw.loc[df_raw["Name"] == key, "Seats"] = value[2]

In [9]:
# 값이 채워짐
df_raw[df_raw["Name"] == "Land Rover Range Rover Sport 2005 2012 Sport"]

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
1460,Land Rover Range Rover Sport 2005 2012 Sport,Coimbatore,62667.4,2008,69078,Petrol,Manual,First,0.0 kmpl,2993.0,241.4,5.0


In [10]:
# Mileage가 0.0 인 값 채워넣기
test = df_raw[df_raw["Mileage"] == "0.0 kmpl"]
test["Name"].unique()

array(['Land Rover Freelander 2 TD4 SE',
       'Mercedes-Benz C-Class Progressive C 220d',
       'Hyundai Santro Xing XL', 'Honda City 1.5 GXI', 'Ford Figo Diesel',
       'Volkswagen Jetta 2007-2011 1.9 L TDI',
       'Hyundai Santro GLS I - Euro I',
       'Mercedes-Benz New C-Class Progressive C 200',
       'Hyundai Santro LS zipPlus', 'Hyundai Santro Xing XP',
       'Mercedes-Benz M-Class ML 350 4Matic',
       'Land Rover Range Rover 3.0 D', 'Smart Fortwo CDI AT',
       'Hyundai Santro Xing GL', 'Land Rover Freelander 2 TD4 S',
       'Hyundai Santro GLS I - Euro II', 'Maruti Baleno Vxi',
       'Land Rover Range Rover Sport 2005 2012 Sport',
       'Hyundai Santro DX', 'Mahindra Jeep MM 550 PE',
       'Hyundai Santro LP zipPlus', 'Toyota Qualis RS E2',
       'Hyundai Santro AT', 'Hyundai Santro GLS II - Euro II',
       'Skoda Superb 3.6 V6 FSI', 'Hyundai Santro GS',
       'Audi A7 2011-2015 Sportback',
       'Hyundai Santro GS zipDrive - Euro II', 'Fiat Siena 1.2 ELX',


In [11]:
dict_Mileage = {
    "Land Rover Freelander 2 TD4 SE" : [12.39],
    "Mercedes-Benz C-Class Progressive C 220d" : [23.0],
    "Hyundai Santro Xing XL" : [12.8],
    "Honda City 1.5 GXI" : [17.8],
    "Ford Figo Diesel" : [25.5],
    "Volkswagen Jetta 2007-2011 1.9 L TDI" : [15.5],
    "Hyundai Santro GLS I - Euro I" : [20.3],
    "Mercedes-Benz New C-Class Progressive C 200" : [16.9], 
    "Hyundai Santro LS zipPlus" : [15.0],
    "Hyundai Santro Xing XP" : [15.0],
    "Mercedes-Benz M-Class ML 350 4Matic" : [7.2],
    "Land Rover Range Rover 3.0 D" : [13.16],
    "Smart Fortwo CDI AT" : [36.59],
    "Hyundai Santro Xing GL" : [17.92],
    "Land Rover Freelander 2 TD4 S" : [17.56],
    "Hyundai Santro GLS I - Euro II" : [30.48],
    "Maruti Baleno Vxi" : [12.0],
    "Land Rover Range Rover Sport 2005 2012 Sport" : [14.7],
    "Hyundai Santro DX" : [20.3],
    "Mahindra Jeep MM 550 PE" : [10.0],
    "Hyundai Santro LP zipPlus" : [None],
    "Toyota Qualis RS E2" : [14.0],
    "Hyundai Santro AT" : [20.3],
    "Hyundai Santro GLS II - Euro II" : [None],
    "Skoda Superb 3.6 V6 FSI" : [5.0],
    "Hyundai Santro GS" : [20.3],
    "Audi A7 2011-2015 Sportback" : [14.4],
    "Hyundai Santro GS zipDrive - Euro II" : [20.3],
    "Fiat Siena 1.2 ELX" : [None],
    "Land Rover Freelander 2 TD4 HSE" : [12.39],
    "Hyundai Santro LP - Euro II" : [None],
    "Toyota Qualis Fleet A3" : [None],
    "Mahindra Jeep MM 540 DP" : [10.0]
    
}
for key, value in dict_Mileage.items():
    df_raw.loc[df_raw["Name"] == key, "Mileage"] = value[0]
    

In [12]:
# Mileage값이 채워짐
df_raw[df_raw["Name"] == "Land Rover Freelander 2 TD4 HSE"]

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
5016,Land Rover Freelander 2 TD4 HSE,Delhi,23760.88,2013,72000,Diesel,Automatic,First,12.39,2179 CC,115 bhp,5.0


In [13]:
df_raw[df_raw['Seats'].isnull()]

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
229,Ford Figo Diesel,Bangalore,5518.66,2015,70436,Diesel,Manual,First,25.5,1498 CC,99 bhp,NaN
1917,Honda City 1.5 EXI,Jaipur,2606.03,2005,88000,Petrol,Manual,Second,13.0 kmpl,1493 CC,100 bhp,NaN
2369,Maruti Estilo LXI,Chennai,2299.44,2008,56000,Petrol,Manual,Second,19.5 kmpl,1061 CC,null bhp,NaN
3800,Ford Endeavour Hurricane LE,Mumbai,10730.72,2012,129000,Diesel,Automatic,First,12.8 kmpl,2953 CC,null bhp,NaN
3882,Maruti Estilo LXI,Kolkata,3832.40,2010,40000,Petrol,Manual,Second,19.5 kmpl,1061 CC,null bhp,NaN
5893,Maruti Estilo LXI,Chennai,2682.68,2008,51000,Petrol,Manual,Second,19.5 kmpl,1061 CC,null bhp,NaN


In [14]:
dict_seat = {
    "Ford Figo Diesel" : [5],
    "Honda City 1.5 EXI" : [5],
    "Maruti Estilo LXI" : [5],
    "Ford Endeavour Hurricane LE" : [7],
    "Endeavour Hurricane LE" : [7]
}
for key, value in dict_seat.items():
    df_raw.loc[df_raw["Name"] == key, "Seats"] = value[0]

In [15]:
# Power가 null bhp인 값을 채우기
test = df_raw[df_raw["Power"] == "null bhp"]
test["Name"].unique()

array(['Ford Fiesta 1.4 SXi TDCi', 'Hyundai Santro Xing XL',
       'Hyundai Santro Xing XO', 'Hyundai Santro Xing XL eRLX Euro III',
       'Hyundai Santro Xing XO eRLX Euro II', 'Maruti Swift VDI BSIV',
       'Volkswagen Jetta 2007-2011 1.9 L TDI',
       'Hyundai Santro GLS I - Euro I', 'Ford Fiesta 1.6 SXI ABS Duratec',
       'Hyundai Santro LS zipPlus', 'Hyundai Santro Xing XP',
       'Tata Indica DLS', 'Toyota Etios Liva G', 'Smart Fortwo CDI AT',
       'Porsche Cayman 2009-2012 S', 'Toyota Etios GD',
       'Toyota Etios Liva GD', 'Hyundai Santro GLS I - Euro II',
       'Fiat Petra 1.2 EL', 'Maruti Baleno Vxi',
       'Chevrolet Optra 1.6 Elite', 'Mahindra Bolero ZLX',
       'Nissan Micra Diesel', 'Hyundai Santro DX',
       'Mercedes-Benz E-Class 220 CDI', 'Mahindra Jeep MM 550 PE',
       'Maruti Swift VDI BSIV W ABS', 'Toyota Qualis RS E2',
       'Porsche Cayman 2009-2012 S tiptronic', 'Hyundai Santro AT',
       'Maruti Estilo LXI', 'Hyundai Santro Xing XO CNG',
     

In [16]:
dict_Power = {
    "Ford Fiesta 1.4 SXi TDCi" : [67.04],
    "Hyundai Santro Xing XL" : [63.0],
    "Hyundai Santro Xing XO" : [63.0],
    "Hyundai Santro Xing XL eRLX Euro III" : [62.1],
    "Hyundai Santro Xing XO eRLX Euro II" : [63.0],
    "Maruti Swift VDI BSIV" : [74.0],
    "Volkswagen Jetta 2007-2011 1.9 L TDI" : [105.0],
    "Hyundai Santro GLS I - Euro I" : [55.0],
    "Ford Fiesta 1.6 SXI ABS Duratec" : [101.0],
    "Hyundai Santro LS zipPlus" : [63.0],
    "Hyundai Santro Xing XP" : [63.0],
    "Tata Indica DLS" : [53.5],
    "Toyota Etios Liva G" : [79.9],
    "Smart Fortwo CDI AT" : [40.0],
    "Porsche Cayman 2009-2012 S" : [320.0],
    "Toyota Etios GD" : [67.0],
    "Toyota Etios Liva GD" : [67.0],
    "Hyundai Santro GLS I - Euro II" : [55.0],
    "Fiat Petra 1.2 EL" : [72.0],
    "Maruti Baleno Vxi" : [82.0],
    "Chevrolet Optra 1.6 Elite" : [104.0],
    "Mahindra Bolero ZLX" : [63,0],
    "Nissan Micra Diesel" : [63.0],
    "Hyundai Santro DX" : [63.0],
    "Mercedes-Benz E-Class 220 CDI" : [170.0],
    "Mahindra Jeep MM 550 PE" : [62.0],
    "Maruti Swift VDI BSIV W ABS" : [74.0],
    "Toyota Qualis RS E2" : [320.0],
    "Porsche Cayman 2009-2012 S tiptronic" : [320.0],
    "Hyundai Santro AT" : [63.0],
    "Maruti Estilo LXI" : [67.0],
    "Hyundai Santro Xing XO CNG" : [62.1],
    "Hyundai Santro Xing XG eRLX Euro III" : [63.0],
    "Hyundai Santro GS" : [63.0],
    "Hyundai Santro GS zipDrive - Euro II" : [63.0],
    "Nissan Teana 230jM" : [157.0],
    "Honda CR-V Sport" : [148.0],
    "Maruti Esteem Vxi" : [85.0],
    "Ford Endeavour Hurricane LE" : [155.0],
    "Audi A4 3.2 FSI Tiptronic Quattro" : [255.0],
    "Maruti Baleno LXI - BSIII" : [94.0],
    "Maruti Esteem LX BSII" : [85.0],
    'Fiat Siena 1.2 ELX' : [80.0],
    'Maruti 1000 AC' : [45.0],
    "Hyundai Santro Xing XL AT eRLX Euro II" : [63.0],
    "Hyundai Santro Xing XL AT eRLX Euro III" : [63.0],
    "Ford Endeavour XLT TDCi 4X4" : [153.0],
    "Hyundai Santro LP - Euro II" : [63.0],
    "Toyota Qualis Fleet A3" : [75.0],
    "Skoda Laura Classic 1.8 TSI" : [160.0],
    'Mahindra Jeep MM 540 DP' : [62.0]
    
}
for key, value in dict_Power.items():
    df_raw.loc[df_raw["Name"] == key, "Power"] = value[0]

In [17]:
# Power값이 채워짐
df_raw[df_raw["Name"] == "Ford Fiesta 1.4 SXi TDCi"]

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
76,Ford Fiesta 1.4 SXi TDCi,Jaipur,3065.92,2008,111111,Diesel,Manual,First,17.8 kmpl,1399 CC,67.04,5.0
3290,Ford Fiesta 1.4 SXi TDCi,Hyderabad,3679.10,2008,83000,Diesel,Manual,Second,17.8 kmpl,1399 CC,67.04,5.0


In [18]:
# Seats가 0인 값 넣기
df_raw[df_raw["Seats"] == 0]

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
3999,Audi A4 3.2 FSI Tiptronic Quattro,Hyderabad,27593.28,2012,125000,Petrol,Automatic,First,10.5 kmpl,3197 CC,255.0,0.0


In [19]:
df_raw.loc[df_raw["Seats"] == 0, "Seats"] = 5

In [20]:
df_raw.isna().sum()

Name                  0
Location              0
Price                 0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage              10
Engine                0
Power                 0
Seats                 0
dtype: int64

In [21]:
# df_raw = df_raw.dropna()
# len(df_raw)
# df_raw
df_raw.shape

(6200, 12)

In [22]:
df_raw['Mileage'] = df_raw['Mileage'].astype(str)
df_raw['Engine'] = df_raw['Engine'].astype(str)
df_raw['Power'] = df_raw['Power'].astype(str)

for i in range(df_raw.shape[0]):
    
    df_raw.at[i, 'Company'] = df_raw['Name'][i].split()[0]
    df_raw.at[i, 'Mileage(km/L)'] = df_raw['Mileage'][i].split()[0]
    df_raw.at[i, 'Engine(CC)'] = df_raw['Engine'][i].split()[0]
    df_raw.at[i, 'Power(bhp)'] = df_raw['Power'][i].split()[0]
    # df_raw.at[i, 'Company'] = df_raw['Name'][i].split()[0]
    # df_raw.at[i, 'Mileage(km/L)'] = df_raw['Mileage'][i].split()[0]
    # df_raw.at[i, 'Engine(CC)'] = df_raw['Engine'][i].split()[0]
    # df_raw.at[i, 'Power(bhp)'] = df_raw['Power'][i].split()[0]
    
df_raw.head()

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Company,Mileage(km/L),Engine(CC),Power(bhp)
0,Maruti Wagon R LXI CNG,Mumbai,2682.68,2010,72000,CNG,Manual,First,26.6 kmpl,998 CC,58.16 bhp,5.0,Maruti,26.6,998,58.16
1,Hyundai Creta 1.6 CRDi SX Option,Pune,19162.00,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,Hyundai,19.67,1582,126.2
2,Honda Jazz V,Chennai,6898.32,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,Honda,18.2,1199,88.7
3,Maruti Ertiga VDI,Chennai,9197.76,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,Maruti,20.77,1248,88.76
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,27194.71,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,Audi,15.2,1968,140.8


In [23]:
df_raw = df_raw.drop(columns=["Engine", "Mileage", "Power"])
# df_raw = df_raw.dropna()
df_raw = df_raw[df_raw['Mileage(km/L)'] != 'None']  


In [24]:
df_raw.reset_index(inplace=False, drop=True)

,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Company,Mileage(km/L),Engine(CC),Power(bhp)
0,Maruti Wagon R LXI CNG,Mumbai,2682.68,2010,72000,CNG,Manual,First,5.0,Maruti,26.6,998,58.16
1,Hyundai Creta 1.6 CRDi SX Option,Pune,19162.00,2015,41000,Diesel,Manual,First,5.0,Hyundai,19.67,1582,126.2
2,Honda Jazz V,Chennai,6898.32,2011,46000,Petrol,Manual,First,5.0,Honda,18.2,1199,88.7
3,Maruti Ertiga VDI,Chennai,9197.76,2012,87000,Diesel,Manual,First,7.0,Maruti,20.77,1248,88.76
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,27194.71,2013,40670,Diesel,Automatic,Second,5.0,Audi,15.2,1968,140.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6187,Tata Tiago 1.05 Revotorq XT Option,Pune,10822.70,2016,41413,Diesel,Manual,First,5.0,Tata,27.28,1047,69
6188,Ford EcoSport 1.5 Petrol Ambiente,Kochi,13474.72,2017,39015,Petrol,Manual,First,5.0,Ford,17.0,1497,121.36
6189,Jeep Compass 1.4 Sport,Mumbai,27838.55,2018,6000,Petrol,Manual,First,5.0,Jeep,16.0,1368,160
6190,Maruti Vitara Brezza ZDi,Kochi,16203.39,2017,21380,Diesel,Manual,First,5.0,Maruti,24.3,1248,88.5


In [25]:
def categorize_price(price):
    Q1 = df['Price'].quantile(0.25)
    Q2 = df['Price'].quantile(0.50)
    Q3 = df['Price'].quantile(0.75)
    IQR = Q3 - Q1
    if price <= Q1:
        return 'c1'
    elif price <= Q2:
        return 'c2'
    elif price <= Q3:
        return 'c3'
    elif price <= Q3 + 1.5 * IQR:
        return 'c4'
    else:
        return 'c5'

In [26]:
df = df_raw

In [27]:
import pandas as pd

# 이름에서 브랜드만 따로 추출
df['Name'] = df['Name'].apply(lambda x: 'Land-Rover' if 'Land Rover' in x else x)
df['Brand'] = df['Name'][:].str.split(' ').str[0]

split_names = df['Name'].str.split(' ')
joined_brands = split_names.apply(lambda x: ' '.join(x[1:]))
df['Name'] = joined_brands


# 브랜드가 속한 나라 별로 딕셔너리 형성
brands = {
    'India': ['Maruti', 'Tata', 'Land-Rover', 'Mahindra', 'Ambassador', 'Force', 'Hindustan'],
    'Japan': ['Honda', 'Toyota', 'Mitsubishi', 'ISUZU', 'Nissan', 'Datsun'],
    'Korea': ['Hyundai'],
    'Germany': ['Volkswagen', 'Mercedes-Benz', 'BMW', 'Porsche', 'Audi', 'Skoda', 'Mini', 'Smart', 'OpelCorsa'],
    'France': ['Renault'],
    'USA': ['Ford', 'Chevrolet', 'Jeep'],
    'UK': ['Jaguar', 'Bentley'],
    'Sweden': ['Volvo'],
    'Italy': ['Fiat', 'Lamborghini']
}

# 새로운 파생 변수 Country 생성 - 모든 데이터들에 결측치 없이 값이 지정됨
brand_country = {brand: country for country, brands_list in brands.items() for brand in brands_list}

df['Country'] = df['Brand'].map(brand_country)


# 나라가 속한 대륙 별로 딕셔너리 형성
continents = {
    'Europe' : ['Germany', 'France', 'Sweden', 'Italy', 'UK'],
    'Asia' : ['Korea', 'Japan', 'India'],
    'America' : ['USA']
}


# 새로운 파생 변수 Continent 생성 - 모든 데이터들에 결측치 없이 값이 지정됨
brand_continent = {}
for continent, countries in continents.items():
    for country in countries:
        brand_continent[country] = continent

# 대륙 정보 기반으로 새로운 열 추가
df['Continent'] = df['Country'].map(brand_continent)

city_population = {
    'Mumbai' : 20961,
    'Hyderabad' : 10534,
    'Coimbatore' : 2935,
    'Kochi' : 3301,
    'Pune' : 6987,
    'Delhi' : 32066,
    'Kolkata' : 15134,
    'Chennai' : 11503,
    'Jaipur' : 4107,
    'Bangalore' :13193,
    'Ahmedabad' : 8450
} # 2022년 기준 인도 도시 별 인구수

df['Price_Cat'] = df['Price'].apply(categorize_price)

df['City_population'] = df['Location'].map(city_population)
# 결과 확인
df.head()


,Name,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Company,Mileage(km/L),Engine(CC),Power(bhp),Brand,Country,Continent,Price_Cat,City_population
0,Wagon R LXI CNG,Mumbai,2682.68,2010,72000,CNG,Manual,First,5.0,Maruti,26.6,998,58.16,Maruti,India,Asia,c1,20961
1,Creta 1.6 CRDi SX Option,Pune,19162.00,2015,41000,Diesel,Manual,First,5.0,Hyundai,19.67,1582,126.2,Hyundai,Korea,Asia,c4,6987
2,Jazz V,Chennai,6898.32,2011,46000,Petrol,Manual,First,5.0,Honda,18.2,1199,88.7,Honda,Japan,Asia,c2,11503
3,Ertiga VDI,Chennai,9197.76,2012,87000,Diesel,Manual,First,7.0,Maruti,20.77,1248,88.76,Maruti,India,Asia,c3,11503
4,A4 New 2.0 TDI Multitronic,Coimbatore,27194.71,2013,40670,Diesel,Automatic,Second,5.0,Audi,15.2,1968,140.8,Audi,Germany,Europe,c4,2935


In [28]:
df.isnull().sum()

Name                 0
Location             0
Price                0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Seats                0
Company              0
Mileage(km/L)        0
Engine(CC)           0
Power(bhp)           0
Brand                0
Country              0
Continent            0
Price_Cat            0
City_population      0
dtype: int64

In [29]:
df = df.drop("Name", axis=1)
df.head()

,Location,Price,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Seats,Company,Mileage(km/L),Engine(CC),Power(bhp),Brand,Country,Continent,Price_Cat,City_population
0,Mumbai,2682.68,2010,72000,CNG,Manual,First,5.0,Maruti,26.6,998,58.16,Maruti,India,Asia,c1,20961
1,Pune,19162.00,2015,41000,Diesel,Manual,First,5.0,Hyundai,19.67,1582,126.2,Hyundai,Korea,Asia,c4,6987
2,Chennai,6898.32,2011,46000,Petrol,Manual,First,5.0,Honda,18.2,1199,88.7,Honda,Japan,Asia,c2,11503
3,Chennai,9197.76,2012,87000,Diesel,Manual,First,7.0,Maruti,20.77,1248,88.76,Maruti,India,Asia,c3,11503
4,Coimbatore,27194.71,2013,40670,Diesel,Automatic,Second,5.0,Audi,15.2,1968,140.8,Audi,Germany,Europe,c4,2935


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6192 entries, 0 to 6199
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           6192 non-null   object 
 1   Price              6192 non-null   float64
 2   Year               6192 non-null   int64  
 3   Kilometers_Driven  6192 non-null   int64  
 4   Fuel_Type          6192 non-null   object 
 5   Transmission       6192 non-null   object 
 6   Owner_Type         6192 non-null   object 
 7   Seats              6192 non-null   float64
 8   Company            6192 non-null   object 
 9   Mileage(km/L)      6192 non-null   object 
 10  Engine(CC)         6192 non-null   object 
 11  Power(bhp)         6192 non-null   object 
 12  Brand              6192 non-null   object 
 13  Country            6192 non-null   object 
 14  Continent          6192 non-null   object 
 15  Price_Cat          6192 non-null   object 
 16  City_population    6192 

In [31]:
df['Mileage(km/L)'] = df['Mileage(km/L)'].astype(float)
df['Engine(CC)'] = df['Engine(CC)'].astype(float)
df['Power(bhp)'] = df['Power(bhp)'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6192 entries, 0 to 6199
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           6192 non-null   object 
 1   Price              6192 non-null   float64
 2   Year               6192 non-null   int64  
 3   Kilometers_Driven  6192 non-null   int64  
 4   Fuel_Type          6192 non-null   object 
 5   Transmission       6192 non-null   object 
 6   Owner_Type         6192 non-null   object 
 7   Seats              6192 non-null   float64
 8   Company            6192 non-null   object 
 9   Mileage(km/L)      6190 non-null   float64
 10  Engine(CC)         6192 non-null   float64
 11  Power(bhp)         6192 non-null   float64
 12  Brand              6192 non-null   object 
 13  Country            6192 non-null   object 
 14  Continent          6192 non-null   object 
 15  Price_Cat          6192 non-null   object 
 16  City_population    6192 

In [32]:
# df.loc[df['Price'] < 10, "Price"]
df = df.drop(df[df['Price'] < 10].index)

In [33]:
df.loc[df['Price'] < 10, "Price"]

Series([], Name: Price, dtype: float64)

In [34]:
df.to_csv('preprocessed_data.csv', index=False)  # index=False로 설정하여 인덱스를 저장하지 않음

print("CSV 파일이 저장되었습니다.")

CSV 파일이 저장되었습니다.
